# Digit Recognizer - MNIST data

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split

In [17]:
# split data into data and labels
train_label = train_data['label'].values.ravel()
train = train_data.loc[:, train_data.columns != 'label']

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(train, train_label, test_size=0.2)

In [23]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(33600, 784)
(33600,)
(8400, 784)
(8400,)


Import machine learning algorithm to use for classification

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import accuracy_score

In [26]:
def train_valid_accuracy(algo, X_train = X_train, y_train = y_train, 
                         X_valid = X_valid, y_valid = y_valid):
    """
    Fit algorithm with training data and return accuracy score with validation data
    """
    train_algo = algo
    model = train_algo.fit(X_train, y_train)
    predict = model.predict(X_valid)
    return accuracy_score(y_valid, predict)

Use `train_valid_accuracy` to get accuracy score for each training algorithm.

In [33]:
print("KNeighborsClassifier:", train_valid_accuracy(KNeighborsClassifier()))
print("RandomForestClassifier:", train_valid_accuracy(RandomForestClassifier()))
print("AdaBoostClassifier:", train_valid_accuracy(AdaBoostClassifier()))
print("SVC:", train_valid_accuracy(SVC()))
print("DecisionTreeClassifier:", train_valid_accuracy(DecisionTreeClassifier()))
print("KMeans:", train_valid_accuracy(KMeans()))
print("GaussianMixture:", train_valid_accuracy(GaussianMixture()))
print("SGDClassifier:", train_valid_accuracy(SGDClassifier()))

KNeighborsClassifier: 0.9669047619047619
RandomForestClassifier: 0.9660714285714286
AdaBoostClassifier: 0.7204761904761905
SVC: 0.9764285714285714
DecisionTreeClassifier: 0.8509523809523809
KMeans: 0.24011904761904762
GaussianMixture: 0.09869047619047619
SGDClassifier: 0.8604761904761905


From the above scores, we see that `KNeightborsClassifier`, `RandomForestClassifier`, and `SVC` has the highest accuracy score. We use `RandomSearchCV` and `GridSearchCV` to find the best parameters for each classification algorithm and get the accuracy score.

In [35]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### KNeighborsClassifier()

In [37]:
param_grid = [{
    'weights': ['uniform', 'distance'], 'n_neighbors': [3, 4, 5]
}]

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3)
grid_search.fit(X_train[:5000], y_train[:5000])

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [3, 4, 5],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [38]:
grid_search.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [39]:
y_pred = grid_search.best_estimator_.predict(X_valid)
print(accuracy_score(y_valid, y_pred))

0.940595238095238


### RandomForestClassifier()

In [41]:
param_grid = [{
    'n_estimators': [100, 150, 200]
}]

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=3)
grid_search.fit(X_train[:5000], y_train[:5000])

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [42]:
grid_search.best_params_

{'n_estimators': 150}

In [43]:
y_pred = grid_search.best_estimator_.predict(X_valid)
print(accuracy_score(y_valid, y_pred))

0.9366666666666666


### SVC()

In [46]:
param_grid = [{
    'C': [1, 5, 10], 'gamma': ['scale', 'auto'], 'tol': [1e-2, 1e-3, 1e-4]
}]

random_search = RandomizedSearchCV(SVC(), param_grid, cv=3, n_jobs=3)
random_search.fit(X_train[:5000], y_train[:5000])

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=3,
                   param_distributions=[{'C': [1, 5, 10],
                                         'gamma': ['scale', 'auto'],
                                         'tol': [0.01, 0.001, 0.0001]}],
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [47]:
random_search.best_params_

{'tol': 0.001, 'gamma': 'scale', 'C': 5}

In [48]:
y_pred = random_search.best_estimator_.predict(X_valid)
print(accuracy_score(y_valid, y_pred))

0.9607142857142857


### Neural Network

We create a neural network model to see if we can get a better accuracy score than the `SVC model`.

In [50]:
import os

In [53]:
model = keras.models.Sequential([
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(784, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(1e-3),
             metrics=["accuracy"])

run_index = 1 
run_logdir = os.path.join(os.curdir, "mnist_logs", "run_{:03d}".format(run_index))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_ann_model_1.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=100, validation_data = (X_valid, y_valid),
                   callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

Epoch 1/100
1050/1050 [==============================] - 1s 1ms/step - loss: 1.2596 - accuracy: 0.6146 - val_loss: 0.8275 - val_accuracy: 0.7367
Epoch 2/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.8122 - accuracy: 0.7507 - val_loss: 0.7264 - val_accuracy: 0.7926
Epoch 3/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.6765 - accuracy: 0.8048 - val_loss: 0.7569 - val_accuracy: 0.7444
Epoch 4/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.5778 - accuracy: 0.8341 - val_loss: 0.5758 - val_accuracy: 0.8410
Epoch 5/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.5265 - accuracy: 0.8485 - val_loss: 0.4931 - val_accuracy: 0.8575
Epoch 6/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.4810 - accuracy: 0.8596 - val_loss: 0.4906 - val_accuracy: 0.8600
Epoch 7/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.4536 - accuracy: 0.8691 - val_loss: 0.4972 - val_ac

In [54]:
model = keras.models.load_model("mnist_ann_model_1.h5")
model.evaluate(X_valid, y_valid)

263/263 [==============================] - 0s 656us/step - loss: 0.2615 - accuracy: 0.9311


[0.2614995241165161, 0.931071400642395]

In [56]:
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(1e-3),
             metrics=["accuracy"])

run_index = 2
run_logdir = os.path.join(os.curdir, "mnist_logs", "run_{:03d}".format(run_index))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_ann_model_2.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=100, validation_data = (X_valid, y_valid),
                   callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

Epoch 1/100
1050/1050 [==============================] - 1s 1ms/step - loss: 1.9583 - accuracy: 0.6291 - val_loss: 0.8581 - val_accuracy: 0.7237
Epoch 2/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.6929 - accuracy: 0.8067 - val_loss: 0.6048 - val_accuracy: 0.8402
Epoch 3/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.5022 - accuracy: 0.8676 - val_loss: 0.4944 - val_accuracy: 0.8702
Epoch 4/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.4095 - accuracy: 0.8894 - val_loss: 0.4563 - val_accuracy: 0.8880
Epoch 5/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.3484 - accuracy: 0.9032 - val_loss: 0.4101 - val_accuracy: 0.8943
Epoch 6/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.3067 - accuracy: 0.9154 - val_loss: 0.3925 - val_accuracy: 0.8938
Epoch 7/100
1050/1050 [==============================] - 1s 1ms/step - loss: 0.2784 - accuracy: 0.9206 - val_loss: 0.3488 - val_ac

In [57]:
model = keras.models.load_model("mnist_ann_model_2.h5")
model.evaluate(X_valid, y_valid)

263/263 [==============================] - 0s 716us/step - loss: 0.2563 - accuracy: 0.9395


[0.2562721073627472, 0.9395238161087036]

In [58]:
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(1e-3),
             metrics=["accuracy"])

run_index = 3
run_logdir = os.path.join(os.curdir, "mnist_logs", "run_{:03d}".format(run_index))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_ann_model_3.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=100, validation_data = (X_valid, y_valid),
                   callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

Epoch 1/100
1050/1050 [==============================] - 2s 2ms/step - loss: 1.7860 - accuracy: 0.7865 - val_loss: 0.5084 - val_accuracy: 0.8648
Epoch 2/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.3953 - accuracy: 0.8885 - val_loss: 0.4006 - val_accuracy: 0.8896
Epoch 3/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2805 - accuracy: 0.9170 - val_loss: 0.3581 - val_accuracy: 0.9114
Epoch 4/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2242 - accuracy: 0.9326 - val_loss: 0.3203 - val_accuracy: 0.9225
Epoch 5/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1848 - accuracy: 0.9433 - val_loss: 0.3194 - val_accuracy: 0.9279
Epoch 6/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1592 - accuracy: 0.9500 - val_loss: 0.2979 - val_accuracy: 0.9295
Epoch 7/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1401 - accuracy: 0.9570 - val_loss: 0.2955 - val_ac

In [59]:
model = keras.models.load_model("mnist_ann_model_3.h5")
model.evaluate(X_valid, y_valid)

263/263 [==============================] - 0s 740us/step - loss: 0.2802 - accuracy: 0.9388


[0.28021055459976196, 0.9388095140457153]

To get a better accuracy score, we try running the model again with different learning rate.

In [60]:
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(0.1),
             metrics=["accuracy"])

run_index = 4
run_logdir = os.path.join(os.curdir, "mnist_logs", "run_{:03d}".format(run_index))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_ann_model_4.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=100, validation_data = (X_valid, y_valid),
                   callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

Epoch 1/100
1050/1050 [==============================] - 1s 1ms/step - loss: 195850240.0000 - accuracy: 0.1077 - val_loss: 2.3013 - val_accuracy: 0.1154
Epoch 2/100
1050/1050 [==============================] - 1s 1ms/step - loss: 2.3024 - accuracy: 0.1096 - val_loss: 2.3016 - val_accuracy: 0.1154
Epoch 3/100
1050/1050 [==============================] - 1s 1ms/step - loss: 2.3020 - accuracy: 0.1082 - val_loss: 2.3008 - val_accuracy: 0.1154
Epoch 4/100
1050/1050 [==============================] - 1s 1ms/step - loss: 2.3021 - accuracy: 0.1086 - val_loss: 2.3007 - val_accuracy: 0.1154
Epoch 5/100
1050/1050 [==============================] - 1s 1ms/step - loss: 2.3022 - accuracy: 0.1093 - val_loss: 2.3014 - val_accuracy: 0.1154
Epoch 6/100
1050/1050 [==============================] - 1s 1ms/step - loss: 2.3021 - accuracy: 0.1067 - val_loss: 2.3001 - val_accuracy: 0.1154
Epoch 7/100
1050/1050 [==============================] - 1s 1ms/step - loss: 2.3022 - accuracy: 0.1086 - val_loss: 2.3017 

In [61]:
model = keras.models.load_model("mnist_ann_model_4.h5")
model.evaluate(X_valid, y_valid)

263/263 [==============================] - 0s 681us/step - loss: 2.3001 - accuracy: 0.1154


[2.3000805377960205, 0.11535714566707611]

Increasing the learning rate results in poor accuracy.

In [69]:
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(1e-3),
             metrics=["accuracy"])

run_index = 5
run_logdir = os.path.join(os.curdir, "mnist_logs", "run_{:03d}".format(run_index))

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_ann_model_5.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=100, validation_data = (X_valid, y_valid),
                   callbacks=[early_stopping_cb, checkpoint_cb, tensorboard_cb])

model.evaluate(X_valid, y_valid)

Epoch 1/100
1050/1050 [==============================] - 2s 2ms/step - loss: 2.0126 - accuracy: 0.5566 - val_loss: 0.9934 - val_accuracy: 0.6987
Epoch 2/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.8057 - accuracy: 0.7580 - val_loss: 0.6586 - val_accuracy: 0.8238
Epoch 3/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.5145 - accuracy: 0.8537 - val_loss: 0.5316 - val_accuracy: 0.8498
Epoch 4/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.3820 - accuracy: 0.8877 - val_loss: 0.4210 - val_accuracy: 0.8931
Epoch 5/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.3139 - accuracy: 0.9058 - val_loss: 0.3750 - val_accuracy: 0.9037
Epoch 6/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2648 - accuracy: 0.9193 - val_loss: 0.3452 - val_accuracy: 0.9131
Epoch 7/100
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2256 - accuracy: 0.9311 - val_loss: 0.3129 - val_ac

[0.3084529638290405, 0.9448809623718262]

We see that changing the amount of neurons in the second layers improves accuracy.

In [55]:
%load_ext tensorboard
%tensorboard --logdir=./mnist_logs --port=6006

By comparing the model we see that the `SVC()` model returns the highest accuracy score despite only using 5000 data to train.

In [71]:
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
random_search.best_params_

{'tol': 0.001, 'gamma': 'scale', 'C': 5}

In [73]:
predict = random_search.best_estimator_.predict(test_data)

In [81]:
submit_pd = pd.DataFrame({"ImageId": list(range(1,len(predict)+1)),
    "Label": predict})
submit_pd.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,7
4,5,3


In [82]:
submit_pd.to_csv('./Data/answer.csv', index=False)